In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import re
import sqlite3
import psycopg2
import json
import os, zipfile
from bs4 import BeautifulSoup

from IPython.core.display import HTML
pd.options.display.max_columns = 100
import datetime
import requests

from sqlalchemy import create_engine
from difflib import SequenceMatcher, get_close_matches
from IPython.core.display import HTML
from ua_parser import user_agent_parser
import seaborn as sb
import pprint
pp = pprint.PrettyPrinter(indent=4)

HTML("<style>.container { width:100% !important; }</style>")

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(open("data_verkiezingsuitslagen/index.htm"), "html.parser")


## Download all ZIP files

In [ ]:
for i, a in enumerate(soup.findAll('a')):
    #if(i>10): break
    link = a.get('href')
    if not re.search('ZIP', link): continue
    full_link = 'https://2019.elections.openknowledge.be/live/' + link
    
    r = requests.get(full_link)
    
    filename = 'data_verkiezingsuitslagen/zipfiles/'+link
    with open(filename, "wb") as handle:
        for data in r.iter_content():
            handle.write(data)

## Unzip all ZIP files

In [ ]:
for file in os.listdir('data_verkiezingsuitslagen/zipfiles/'):
    if not re.search('ZIP', file): continue
    print('Unzizipping {}'.format(file))
    zip_ref = zipfile.ZipFile('data_verkiezingsuitslagen/zipfiles/' + file, 'r')
    zip_ref.extractall('data_verkiezingsuitslagen/xmls/')
    zip_ref.close()

## Understand filenames

- Enkel files van de Kamer: CK
- Geen tussentijdse bestanden, dus geen _[0-9][0-9][0-9]_ in de filename, zie handleiding
- Dit zijn allemaal bestanden van Kanton hoofdbureaus

- Enkel NL bestanden, geen tweetalige bestanden. _NL.xml in de filename, en niet _NL_FR.xml of _FR_NL.xml

In [ ]:
interesting_files = []
for file in os.listdir('data_verkiezingsuitslagen/xmls/'):
    if not re.search('EML', file): continue
    #if re.search('_NL_FR.EML', file) or re.search('_FR_NL.EML', file): continue
    if not re.search('_CK', file): continue
    if  re.search('_[0-9][0-9][0-9]_', file): continue
    print(file)
    interesting_files.append(file)

## Loop over files

In [ ]:
def parse_kamer_file(filename):
    soup = BeautifulSoup(open(filename), "html.parser")
    contestname = soup.find('contestname').text
    niscode = soup.find('contestidentifier').attrs['idnumber']
    
    candidates = []
    partijen = []

    for reportingunitvotes in soup.findAll('reportingunitvotes'):
        partij = reportingunitvotes.reportingunitidentifier.text

        for i, selection in enumerate(soup.findAll('selection')):
            if selection.find('registeredname'): 
                #print(selection)
                temp_dict = {'name':selection.registeredname.text, 
                                'validvotes':selection.validvotes.text, 
                            'niscode':niscode,
                            'kanton':contestname}
                for countmetric in selection.findAll('countmetric'):
                    temp_dict[countmetric.attrs['type']] = countmetric.text
                partijen.append(temp_dict)

                continue
            elif selection.find('candidate'):
                candidates.append({'name': selection.candidate.knownas.text, 
                                  'votes': selection.validvotes.text, 
                                  'partij':partij, 
                                  'niscode': niscode, 
                                  'kanton': contestname})
            #else:
                #print(selection)

    # Zitten blijkbaar dubbels in de partijen, elke record zit er 10 keer in?
    partijen = pd.DataFrame(partijen).drop_duplicates().to_dict(orient='records')
    candidates = pd.DataFrame(candidates).drop_duplicates().to_dict(orient='records')    
                
    return candidates, partijen

In [ ]:
candidates_total = []
partijen_total = []
for i, file in enumerate(interesting_files):
    print(i)
    candidates, partijen = parse_kamer_file('data_verkiezingsuitslagen/xmls/{}'.format(file))
    candidates_total.extend(candidates)
    partijen_total.extend(partijen)



## Convert to CSV and save

In [ ]:
candidates_total = pd.DataFrame(candidates_total)
candidates_total.to_csv('data_verkiezingsuitslagen/candidates_total.csv', index=False)

In [ ]:
partijen_total = pd.DataFrame(partijen_total)
partijen_total.to_csv('data_verkiezingsuitslagen/partijen_total.csv', index=False)